## 데이터 및 라이브러리 로드

In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *
import tensorflow_addons as tfa

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
%%time
# 약 20초 걸림
with open('D:/inputs/24/train50.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y50.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)


with open('D:/inputs/24/test.pickle', 'rb') as f:
    TEST = pickle.load(f)
TEST = TEST[:, :, :, :10]    
# 아웃라이어 수정
# train[1,16, 24, -2] = 163.05731201

## 회전변환

In [ ]:
## 회전변환시 마이너스(-) 하는 경우, 제곱을 해주면 target값과의 상관관계가 높아짐. 테스트해볼것!
# v1_m_h1 = ((train[:, :, :, 0] * np.cos(np.pi / 4) - train[:, :, :, 1] * np.sin(np.pi / 4)) ** 0.5) / 100

In [ ]:
v1_m_h1 = train[:, :, :, 0] * np.cos(np.pi / 4) - train[:, :, :, 1] * np.sin(np.pi / 4)  
v1_p_h1 = train[:, :, :, 0] * np.cos(np.pi / 4) + train[:, :, :, 1] * np.sin(np.pi / 4)  
train[:, :, :, 0] = v1_m_h1
train[:, :, :, 1] = v1_p_h1 
del v1_m_h1
del v1_p_h1

v2_m_h2 = train[:, :, :, 2] * np.cos(np.pi / 4) - train[:, :, :, 3] * np.sin(np.pi / 4)
v2_p_h2 = train[:, :, :, 2] * np.cos(np.pi / 4) + train[:, :, :, 3] * np.sin(np.pi / 4)
train[:, :, :, 2] = v2_m_h2
train[:, :, :, 3] = v2_p_h2 
del v2_m_h2
del v2_p_h2


In [ ]:
v1_m_h1 = TEST[:, :, :, 0] * np.cos(np.pi / 4) - TEST[:, :, :, 1] * np.sin(np.pi / 4)  
v1_p_h1 = TEST[:, :, :, 0] * np.cos(np.pi / 4) + TEST[:, :, :, 1] * np.sin(np.pi / 4)  
TEST[:, :, :, 0] = v1_m_h1
TEST[:, :, :, 1] = v1_p_h1 
del v1_m_h1
del v1_p_h1

v2_m_h2 = TEST[:, :, :, 2] * np.cos(np.pi / 4) - TEST[:, :, :, 3] * np.sin(np.pi / 4)
v2_p_h2 = TEST[:, :, :, 2] * np.cos(np.pi / 4) + TEST[:, :, :, 3] * np.sin(np.pi / 4)
TEST[:, :, :, 2] = v2_m_h2
TEST[:, :, :, 3] = v2_p_h2 
del v2_m_h2
del v2_p_h2

In [ ]:
v4_p_h4_30 = train[:, :, :, 5] * np.cos(np.pi / 6) + train[:, :, :, 6] * np.sin(np.pi / 6)
v4_m_h4_30 = train[:, :, :, 5] * np.cos(np.pi / 6) - train[:, :, :, 6] * np.sin(np.pi / 6)
train[:, :, :, 5] = v4_p_h4_30
train[:, :, :, 6] = v4_m_h4_30

v5_p_h5_30 = train[:, :, :, 7] * np.cos(np.pi / 6) + train[:, :, :, 8] * np.sin(np.pi / 6)
v5_m_h5_30 = train[:, :, :, 7] * np.cos(np.pi / 6) - train[:, :, :, 8] * np.sin(np.pi / 6)
train[:, :, :, 7] = v5_p_h5_30
train[:, :, :, 8] = v5_m_h5_30

del v4_p_h4_30
del v4_m_h4_30
del v5_p_h5_30
del v5_m_h5_30

In [ ]:
v4_p_h4_30 = TEST[:, :, :, 5] * np.cos(np.pi / 6) + TEST[:, :, :, 6] * np.sin(np.pi / 6)
v4_m_h4_30 = TEST[:, :, :, 5] * np.cos(np.pi / 6) - TEST[:, :, :, 6] * np.sin(np.pi / 6)
TEST[:, :, :, 5] = v4_p_h4_30
TEST[:, :, :, 6] = v4_m_h4_30

v5_p_h5_30 = TEST[:, :, :, 7] * np.cos(np.pi / 6) + TEST[:, :, :, 8] * np.sin(np.pi / 6)
v5_m_h5_30 = TEST[:, :, :, 7] * np.cos(np.pi / 6) - TEST[:, :, :, 8] * np.sin(np.pi / 6)
TEST[:, :, :, 7] = v5_p_h5_30
TEST[:, :, :, 8] = v5_m_h5_30

del v4_p_h4_30
del v4_m_h4_30
del v5_p_h5_30
del v5_m_h5_30

In [ ]:
# 4개의 경도, 위도 정보 삭제함.
train = train[:, :, :, :10]

In [ ]:
# 4개의 경도, 위도 정보 삭제함.
TEST = TEST[:, :, :, :10]

In [ ]:
train.shape

In [ ]:
TEST.shape

In [ ]:
from sklearn.model_selection import train_test_split
train, test, train_y, test_y = train_test_split(train, train_y, test_size=0.05, random_state=SEED)

In [ ]:
train = np.concatenate([train, train_y], -1)

In [ ]:
%%time

train1 = np.rot90(train, 1, (1,2))
train2 = np.rot90(train, 2, (1,2))
train3 = np.rot90(train, 3, (1,2))
train_lr = np.fliplr(train)
# train_ud = np.flipud(train)

train = np.vstack([train, train1])
del train1

train = np.vstack([train, train2])
del train2

train = np.vstack([train, train3])
del train3

train = np.vstack([train, train_lr])
del train_lr

# train = np.vstack([train, train_ud])
# del train_ud

In [ ]:
train_y = train[:, :, :, -1].copy()
train_y = train_y.reshape(train_y.shape[0], train_y.shape[1], train_y.shape[2], 1)

In [ ]:
train = train[:,:,:,:-1]

In [ ]:
from sklearn.model_selection import train_test_split
train, test, train_y, test_y = train_test_split(train, train_y, test_size=0.025, random_state=SEED)

In [ ]:
from models import build_model, build_model2
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [ ]:
def resnet_model():
    inputs=Input(train.shape[1:])
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
model = resnet_model()
model.compile(loss="mae", optimizer="adam", metrics=["mae"])
model.load_weights(f'best_0.h5')

In [ ]:
tmp = model.predict(test)
maeOverFscore(test_y, np.exp(tmp)-1)

In [ ]:
6.2
6.1

1.4658718777229207
1.4790608